In [1]:
import numpy as np
import numpy.random as npr
from scipy.stats import norm
from keras.layers import Input, Dense, Concatenate, SimpleRNN, LSTM, GRU
from keras.models import Model
from keras import backend as kb
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.experimental.numpy as tfnp
import copy
plt.style.use('ggplot')
%run Fourier.ipynb #imports Heston option prices
%run Path_simulation.ipynb #imports path simulation functions
%run Loss_functions.ipynb #imports Loss_functions used
%run Neural_Networks.ipynb #imports different types of NN
%run Analytic_Heston.ipynb #imports function for S2 price calculation and analytic deltas.

Exception: File `'Fourier.ipynb.py'` not found.

In [2]:
#parameters used - mainly from Buehler Deep Hedging paper.

#¤What about feller condition? 2\kappa\theta>\sigma^2 for V_t>0 for all t\in[0,T]
kappa = 1#rate of mean reversion
theta = 0.04#long term variance
rho = -0.7#correlation b/t BM
sigma = 0.2#vol of vol -¤sigma=2 in paper, but maybe it should be 0.2 b/c of Feller condition
V0 = 0.04#initial vol
r = 0

S0 = 100#initial price
n = 100000#number of paths
K = 100#strike
divyield=0
steps = 30#number of rebalancings
epochs = 50#Number of times model is trained
#new variables
T = 30/365#time to expiry
dt = T/steps #step size

#Expected shortfall at level 1-alpha
alpha=0.99

#Transaction cost
TC=False
epsilon=0.01

#Consider calling the epsilon variable, sigma, instead
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

In [3]:
#Data simulation
S1,vol,S2 = HestonFullTruncation_SV(n)
tSVinput_FFNN,dSVoutput_FFNN = DataInputOutput_HSV(S1,vol,S2) 
tSVinput_RNN,dSVoutput_RNN = RNN_DataInputOutput_HSV(S1,vol,S2) 

In [4]:
#No TC
TC=False

In [5]:
#FFNN 
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 8s 5ms/step - loss: 2.9186
Epoch 2/50
1000/1000 [==============================] - 5s 5ms/step - loss: 2.1449
Epoch 3/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.4641
Epoch 4/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.4044
Epoch 5/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.3489
Epoch 6/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.2683
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1694
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.9708
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6875
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.4016
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3228
Epoch 12/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2938
E

In [6]:
model.save('Models/FFNN_NoTC')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC\assets


In [7]:
#RNN
model=RNN_final('linear',3,2,loss_call_RNN)
model.fit(tSVinput_RNN,dSVoutput_RNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 14s 13ms/step - loss: 4.2955
Epoch 2/50
1000/1000 [==============================] - 12s 12ms/step - loss: 2.7841
Epoch 3/50
1000/1000 [==============================] - 13s 13ms/step - loss: 2.0637
Epoch 4/50
1000/1000 [==============================] - 12s 12ms/step - loss: 1.6709
Epoch 5/50
1000/1000 [==============================] - 12s 12ms/step - loss: 1.5328
Epoch 6/50
1000/1000 [==============================] - 12s 12ms/step - loss: 1.4707
Epoch 7/50
1000/1000 [==============================] - 13s 13ms/step - loss: 1.4133
Epoch 8/50
1000/1000 [==============================] - 13s 13ms/step - loss: 1.3539
Epoch 9/50
1000/1000 [==============================] - 15s 15ms/step - loss: 1.2640
Epoch 10/50
1000/1000 [==============================] - 18s 18ms/step - loss: 1.0943
Epoch 11/50
1000/1000 [==============================] - 18s 18ms/step - loss: 0.8657
Epoch 12/50
1000/1000 [==============================] - 18s 18

In [8]:
model.save('Models/RNN_NoTC')

INFO:tensorflow:Assets written to: Models/RNN_NoTC\assets


INFO:tensorflow:Assets written to: Models/RNN_NoTC\assets


In [13]:
#TC 
TC=True
epsilon=0.01#consider trying different TC

In [19]:
callprice=4.0#This is the best guess of a callprice in this setup. Maybe it will help the network converge faster.

In [20]:
#FFNN 
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 8s 6ms/step - loss: 3.6272
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8267
Epoch 3/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.6877
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.6259
Epoch 5/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.6000
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.5474
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.4721
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.4174
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.2425
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.0373
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.7544
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.5622
E

In [21]:
model.save('Models/FFNN_TC_WTerminalC')

INFO:tensorflow:Assets written to: Models/FFNN_TC_WTerminalC\assets


INFO:tensorflow:Assets written to: Models/FFNN_TC_WTerminalC\assets


In [22]:
#RNN
model=RNN_final('linear',3,2,loss_call_RNN)
model.fit(tSVinput_RNN,dSVoutput_RNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 17s 14ms/step - loss: 113.4972
Epoch 2/50
1000/1000 [==============================] - 15s 15ms/step - loss: 3.1979
Epoch 3/50
1000/1000 [==============================] - 14s 14ms/step - loss: 3.1244
Epoch 4/50
1000/1000 [==============================] - 14s 14ms/step - loss: 2.9674 0s - loss: 2.9
Epoch 5/50
1000/1000 [==============================] - 13s 13ms/step - loss: 2.8224
Epoch 6/50
1000/1000 [==============================] - 13s 13ms/step - loss: 2.4119
Epoch 7/50
1000/1000 [==============================] - 14s 14ms/step - loss: 2.0266
Epoch 8/50
1000/1000 [==============================] - 14s 14ms/step - loss: 1.9127
Epoch 9/50
1000/1000 [==============================] - 13s 13ms/step - loss: 1.7924
Epoch 10/50
1000/1000 [==============================] - 13s 13ms/step - loss: 1.7912
Epoch 11/50
1000/1000 [==============================] - 14s 14ms/step - loss: 1.6415
Epoch 12/50
1000/1000 [=======================

In [23]:
model.save('Models/RNN_TC_WTerminalC')

INFO:tensorflow:Assets written to: Models/RNN_TC_WTerminalC\assets


INFO:tensorflow:Assets written to: Models/RNN_TC_WTerminalC\assets


# Stress testing

In [6]:
TC=False
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

## Feller not true

### Sigma=2

In [17]:
sigma=2
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

In [18]:
#Data simulation
S1,vol,S2 = HestonMomentMatching_SV(n)
tSVinput_FFNN,dSVoutput_FFNN = DataInputOutput_HSV(S1,vol,S2) 

In [19]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 9s 6ms/step - loss: 2.8072
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 2.5934
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.9151
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.7252
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.7023
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.6453
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.5065
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.4874
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.4026
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.3155
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.0258
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6201
E

In [20]:
model.save('Models/FFNN_NoTC_sigma2_50e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma2_50e\assets


In [21]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2903
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2876A: 0
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2886
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2932
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2777
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2763A: 0s - loss:
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2782
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2727
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2662
Epoch 10/50
1000/1000 [==============================] - 6s 5ms/step - loss: 0.2670
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2616
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step

In [22]:
model.save('Models/FFNN_NoTC_sigma2_100e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma2_100e\assets


In [23]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1808
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1832
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1683
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1748A: 0s - loss
Epoch 5/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1710
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1765
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1670
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1695
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1701
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1695
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1621
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - lo

In [24]:
model.save('Models/FFNN_NoTC_sigma2_150e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma2_150e\assets


In [25]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1502
Epoch 2/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1480
Epoch 3/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1495
Epoch 4/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1533
Epoch 5/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1480
Epoch 6/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1563
Epoch 7/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1480
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1473
Epoch 9/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1464
Epoch 10/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1470
Epoch 11/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1535
Epoch 12/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1443
E

In [26]:
model.save('Models/FFNN_NoTC_sigma2_200e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma2_200e\assets


### Sigma=4

In [30]:
sigma=4
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

In [31]:
#Data simulation
S1,vol,S2 = HestonMomentMatching_SV(n)
tSVinput_FFNN,dSVoutput_FFNN = DataInputOutput_HSV(S1,vol,S2) 

In [32]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 8s 6ms/step - loss: 3.3059
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 2.6566
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.9790
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.7343
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.6950
Epoch 6/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.7691
Epoch 7/50
1000/1000 [==============================] - 8s 8ms/step - loss: 1.6582
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.5411
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.5307
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.4066
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.4434
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.3686
E

In [33]:
model.save('Models/FFNN_NoTC_sigma4_50e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma4_50e\assets


In [34]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3850
Epoch 2/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3676
Epoch 3/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3695
Epoch 4/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3595
Epoch 5/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3523
Epoch 6/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3568
Epoch 7/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3460
Epoch 8/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3426
Epoch 9/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.3398
Epoch 10/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.3327
Epoch 11/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.3181
Epoch 12/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.3263
E

In [35]:
model.save('Models/FFNN_NoTC_sigma4_100e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma4_100e\assets


In [36]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2141
Epoch 2/50
1000/1000 [==============================] - 10s 10ms/step - loss: 0.2126
Epoch 3/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1995
Epoch 4/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2090
Epoch 5/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2157
Epoch 6/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2046
Epoch 7/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2089
Epoch 8/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2004
Epoch 9/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2057
Epoch 10/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2063
Epoch 11/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1959
Epoch 12/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1998

In [37]:
model.save('Models/FFNN_NoTC_sigma4_150e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma4_150e\assets


In [38]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1891
Epoch 2/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1622
Epoch 3/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1620
Epoch 4/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1655
Epoch 5/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1741
Epoch 6/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1626
Epoch 7/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1685
Epoch 8/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1618
Epoch 9/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1854
Epoch 10/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1584
Epoch 11/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1644
Epoch 12/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1698
E

In [39]:
model.save('Models/FFNN_NoTC_sigma4_200e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma4_200e\assets


### Sigma=6

In [5]:
sigma=6
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

In [6]:
#Data simulation
S1,vol,S2 = HestonMomentMatching_SV(n)
tSVinput_FFNN,dSVoutput_FFNN = DataInputOutput_HSV(S1,vol,S2) 

In [7]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 8s 6ms/step - loss: 3.7148
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 3.0516
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 2.1439
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8149
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.8707
Epoch 6/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.6630
Epoch 7/50
1000/1000 [==============================] - 8s 8ms/step - loss: 1.5673
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.6838
Epoch 9/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.4683
Epoch 10/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.4349
Epoch 11/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.4424
Epoch 12/50
1000/1000 [==============================] - 7s 7ms/step - loss: 1.5384
E

In [8]:
model.save('Models/FFNN_NoTC_sigma6_50e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma6_50e\assets


In [9]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6476
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.5473
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6868
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.5880
Epoch 5/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6299
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6744
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6266
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.6480
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6128
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6407
Epoch 11/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.5624
Epoch 12/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.5571
E

In [10]:
model.save('Models/FFNN_NoTC_sigma6_100e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma6_100e\assets


In [11]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3669
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3557
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3411
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3518
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3375
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3269
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3248
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3256
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3216
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3108
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3048
Epoch 12/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.2981
E

In [12]:
model.save('Models/FFNN_NoTC_sigma6_150e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma6_150e\assets


In [13]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2182
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2112
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1922
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2048
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2002
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1944
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2030
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2080
Epoch 9/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2098
Epoch 10/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1954
Epoch 11/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2004
Epoch 12/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1943
E

In [14]:
model.save('Models/FFNN_NoTC_sigma6_200e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma6_200e\assets


### Sigma=0.2 for comparison

In [21]:
sigma=0.2
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

In [22]:
#Data simulation
S1,vol,S2 = HestonMomentMatching_SV(n)
tSVinput_FFNN,dSVoutput_FFNN = DataInputOutput_HSV(S1,vol,S2) 

In [23]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 9s 6ms/step - loss: 3.1248
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 2.3907
Epoch 3/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.5010
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.4610
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.3728
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.2871
Epoch 7/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.2220
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.0180
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.7219
Epoch 10/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4480
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3779
Epoch 12/50
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2904
E

In [24]:
model.save('Models/FFNN_NoTC_sigma02_50e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma02_50e\assets


In [25]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1867
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1853
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1746
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1745
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1797
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1721
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1756
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1785
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1706
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1810
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1780
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1697
E

In [26]:
model.save('Models/FFNN_NoTC_sigma02_100e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma02_100e\assets


In [27]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 7s 6ms/step - loss: 0.1621
Epoch 2/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1604
Epoch 3/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1602
Epoch 4/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1650
Epoch 5/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1603
Epoch 6/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1643
Epoch 7/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1623
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1582
Epoch 9/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1627
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1603
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1621
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1559
E

In [28]:
model.save('Models/FFNN_NoTC_sigma02_150e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma02_150e\assets


In [29]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1558
Epoch 2/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1542
Epoch 3/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1562
Epoch 4/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1574
Epoch 5/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1560
Epoch 6/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1548
Epoch 7/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1586
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1498
Epoch 9/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1566
Epoch 10/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.1536
Epoch 11/50
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1533
Epoch 12/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1568
E

In [30]:
model.save('Models/FFNN_NoTC_sigma02_200e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_sigma02_200e\assets


In [31]:
#reset sigma parameter
sigma=0.2
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

## Strike different from S0. We try 80 and 120 as a little data is avaiable above/below here

In [16]:
K=85
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

In [11]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 8s 6ms/step - loss: 1.0381
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0510
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0456
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0458
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0457
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0403
Epoch 7/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0493
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0437
Epoch 9/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0423
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0443
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0495
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0452
E

In [12]:
model.save('Models/FFNN_NoTC_K85')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_K85\assets


In [17]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=400)

Epoch 1/400
1000/1000 [==============================] - 9s 6ms/step - loss: 0.6175
Epoch 2/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0498
Epoch 3/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0427
Epoch 4/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0449
Epoch 5/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0403
Epoch 6/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0431
Epoch 7/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0387
Epoch 8/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0455
Epoch 9/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0438
Epoch 10/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0410
Epoch 11/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0361
Epoch 12/400
1000/1000 [==============================] - 6s 6ms/step - lo

1000/1000 [==============================] - 7s 7ms/step - loss: 0.0195
Epoch 192/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0230
Epoch 193/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0258
Epoch 194/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0253
Epoch 195/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0233
Epoch 196/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0224
Epoch 197/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0240
Epoch 198/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0217
Epoch 199/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0195
Epoch 200/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0204
Epoch 201/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0187
Epoch 202/400
1000/1000 [==============================] - 6s 6ms/st

1000/1000 [==============================] - 5s 5ms/step - loss: 0.0137
Epoch 381/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0157
Epoch 382/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0147
Epoch 383/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0346
Epoch 384/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0394
Epoch 385/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0303
Epoch 386/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0210
Epoch 387/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0129
Epoch 388/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0140
Epoch 389/400
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0165
Epoch 390/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0153
Epoch 391/400
1000/1000 [==============================] - 6s 6ms/st

In [18]:
model.save('Models/FFNN_NoTC_K85_400e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_K85_400epochs\assets


In [19]:
K=115
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

In [14]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 8s 6ms/step - loss: 0.1060
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0130
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0138
Epoch 4/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0164
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0114
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0131
Epoch 7/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0124
Epoch 8/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0098
Epoch 9/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0113
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0123
Epoch 11/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0119
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0125
E

In [15]:
model.save('Models/FFNN_NoTC_K115')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_K115\assets


In [20]:
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=400)

Epoch 1/400
1000/1000 [==============================] - 8s 6ms/step - loss: 0.0488
Epoch 2/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0149
Epoch 3/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0113
Epoch 4/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0113
Epoch 5/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0129
Epoch 6/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0105
Epoch 7/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0113
Epoch 8/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0137
Epoch 9/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0123
Epoch 10/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0128
Epoch 11/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0098
Epoch 12/400
1000/1000 [==============================] - 6s 6ms/step - lo

1000/1000 [==============================] - 6s 6ms/step - loss: 0.0103
Epoch 193/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0102
Epoch 194/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0085
Epoch 195/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0107
Epoch 196/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0104
Epoch 197/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0130
Epoch 198/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0101
Epoch 199/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0099
Epoch 200/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0122
Epoch 201/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0121
Epoch 202/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0115
Epoch 203/400
1000/1000 [==============================] - 6s 6ms/st

1000/1000 [==============================] - 6s 6ms/step - loss: 0.0112
Epoch 383/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0120
Epoch 384/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0109
Epoch 385/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0105
Epoch 386/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0109
Epoch 387/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0126
Epoch 388/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0097
Epoch 389/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0120
Epoch 390/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0132
Epoch 391/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0125
Epoch 392/400
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0117
Epoch 393/400
1000/1000 [==============================] - 6s 6ms/st

In [21]:
model.save('Models/FFNN_NoTC_K115_400e')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_K115_400e\assets


In [22]:
#reset strike price
K=100
callprice = HestonFourier(spot=S0, timetoexp=T, strike=K, r=0, divyield=0, V=V0, theta=theta, kappa=kappa, epsilon=sigma, rho=rho, greek=1)

## Digital option

In [8]:
digiprice = np.mean(S1.T[steps]>K)#this MC pricing should be under the Q measure. 

In [9]:
model=FFNN('linear',3,2,loss_digi_call)
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=200)

Epoch 1/200
1000/1000 [==============================] - 7s 5ms/step - loss: 0.1357
Epoch 2/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0928
Epoch 3/200
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0912
Epoch 4/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0914
Epoch 5/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0908
Epoch 6/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0905
Epoch 7/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0907
Epoch 8/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0908
Epoch 9/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0901
Epoch 10/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0902
Epoch 11/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0850
Epoch 12/200
1000/1000 [==============================] - 5s 5ms/step - lo

1000/1000 [==============================] - 6s 6ms/step - loss: 0.0577
Epoch 193/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0575
Epoch 194/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0573
Epoch 195/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0479
Epoch 196/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0471
Epoch 197/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0432
Epoch 198/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0421
Epoch 199/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0403
Epoch 200/200
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0393


In [11]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0391
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0380
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0393
Epoch 4/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0379
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0377
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0371
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0385
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0374
Epoch 9/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0382
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0367
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0373
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0373
E

In [13]:
model.fit(tSVinput_FFNN,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0365
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0358
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0368
Epoch 4/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0352
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0360
Epoch 6/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0360
Epoch 7/50
1000/1000 [==============================] - 8s 8ms/step - loss: 0.0356
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0356
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0356
Epoch 10/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.0355
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0357
Epoch 12/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0355
E

In [14]:
model.save('Models/FFNN_NoTC_digi')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_digi\assets


## Noise on variance

In [12]:
tSVinput_FFNN_error=copy.deepcopy(tSVinput_FFNN)
Error_term=(1+npr.normal(0,np.sqrt(0.1),(steps,n)))
Error_term[Error_term<0]=0
for i in range(steps):
    for j in range(n):
        tSVinput_FFNN_error[i][j][2]*=Error_term[i][j]

In [13]:
#FFNN 
model=FFNN('linear',3,2,loss_call)
model.fit(tSVinput_FFNN_error,dSVoutput_FFNN,batch_size=100,epochs=50)

Epoch 1/50
1000/1000 [==============================] - 9s 6ms/step - loss: 3.1510
Epoch 2/50
1000/1000 [==============================] - 6s 6ms/step - loss: 2.6451A: 0s - loss:
Epoch 3/50
1000/1000 [==============================] - 6s 6ms/step - loss: 2.4492
Epoch 4/50
1000/1000 [==============================] - 7s 7ms/step - loss: 2.3705
Epoch 5/50
1000/1000 [==============================] - 6s 6ms/step - loss: 2.1273
Epoch 6/50
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1600
Epoch 7/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3857
Epoch 8/50
1000/1000 [==============================] - 7s 7ms/step - loss: 0.3254
Epoch 9/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3065
Epoch 10/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2764
Epoch 11/50
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2652
Epoch 12/50
1000/1000 [==============================] - ETA: 0s - loss:

In [14]:
model.save('Models/FFNN_NoTC_Noisy_Variance')

INFO:tensorflow:Assets written to: Models/FFNN_NoTC_Noisy_Variance\assets
